In [1]:
from Fetcher import Dataset

In [2]:
d = Dataset()

In [3]:
res = d.download("Miami", "wind")

In [4]:
d.save(res)

wind/Miami.json Saved.


{'data': {'1989-12-31': 8.950067718153383,
  '1990-01-01': 10.634594929290555,
  '1990-01-02': 15.61722157215224,
  '1990-01-03': 14.478974786818506,
  '1990-01-04': 15.333873822256253,
  '1990-01-05': 11.296619146809908,
  '1990-01-06': 9.687129306388297,
  '1990-01-07': 10.723566731700268,
  '1990-01-08': 11.583332216239725,
  '1990-01-09': 5.386591906840331,
  '1990-01-10': 8.236365810763447,
  '1990-01-11': 7.142754081006604,
  '1990-01-12': 10.634875672755387,
  '1990-01-13': 18.04965344506908,
  '1990-01-14': 14.305640164108807,
  '1990-01-15': 14.65659306767457,
  '1990-01-16': 17.878358281692506,
  '1990-01-17': 15.878016010441032,
  '1990-01-18': 12.47125026223078,
  '1990-01-19': 13.934049698530666,
  '1990-01-20': 14.80304754775108,
  '1990-01-21': 9.971328947313962,
  '1990-01-22': 4.157899382210821,
  '1990-01-23': 7.4348742347333285,
  '1990-01-24': 12.157569125044313,
  '1990-01-25': 13.193118924010724,
  '1990-01-26': 16.107042485506827,
  '1990-01-27': 15.1814606983116